In [10]:
from ortools.linear_solver import pywraplp
from dataclasses import dataclass
import pandas as pd   
from datetime import datetime
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import threading
import random
import concurrent
from packaging import version

if version.parse(pd.__version__) < version.parse("1.4.0"):
    print("Pandas 1.2 required")

Pandas 1.2 required


## Load Pods


In [11]:
pods_data_src = pd.read_csv("DEV_10_dec - pods.csv")

pd.to_numeric(pods_data_src['req_mem_byte']) / 1000000

pods_data_src['req_cpu_milli_core'] = pd.to_numeric(pods_data_src['req_cpu_milli_core'])

pods_data_src['req_mem_mb'] = pd.to_numeric(pods_data_src['req_mem_byte']) / 1000000


pods_data = pods_data_src[(pods_data_src.node_group == 3) &  
                          (pods_data_src.owner_kind != 'DaemonSet') & 
                          (pods_data_src.req_cpu_milli_core>0) &
                          (pods_data_src.req_mem_byte>0) ][['namespace', 'owner_name', 'req_mem_mb', 'req_cpu_milli_core' ]]

daemonset = pods_data_src[(pods_data_src.node_group == 3) & (pods_data_src.owner_kind == 'DaemonSet')].\
            groupby([ "owner_name", "namespace"]).agg({'req_cpu_milli_core':'mean', 'req_mem_mb':'mean'})

overhead = {'cpu': daemonset.req_cpu_milli_core.sum(), 'memory': daemonset.req_mem_mb.sum()}




## Load Nodes

In [12]:
import re

#regex to clean out any text after the number
def clean_data(value):
    return (re.sub(r'^([0-9]+).*', '\\1', value))


In [13]:
nodes_data = pd.read_csv("instances.csv")

nodes_data.vCPUs = nodes_data.vCPUs.apply(clean_data)

nodes_data['cpu']  = pd.to_numeric(nodes_data.vCPUs, errors='coerce')*1000

nodes_data['memory'] = pd.to_numeric(nodes_data.Memory, errors='coerce')*1000

nodes_data['cost'] =  pd.to_numeric(nodes_data['Linux Reserved cost'], errors='coerce')

nodes_data = nodes_data[['API Name','Name','cpu', 'memory', 'cost']]


total_node_types = len(nodes_data)

nodes_data = nodes_data[(nodes_data.cpu >= pods_data.req_cpu_milli_core.max()) & (nodes_data.memory >= pods_data.req_mem_mb.max()) ]

suitable_node_types = len(nodes_data)

nodes_data.cpu = nodes_data.cpu - overhead['cpu']
nodes_data.memory = nodes_data.memory - overhead['memory']

print(f"Detected {suitable_node_types} suitable nodes out of {total_node_types} total")


Detected 310 suitable nodes out of 382 total


In [14]:
#Create a data model
#
def create_data_model(cpu, memory, pods):
    """Create the data for the example."""
    data = {}    
    data['req_cpu'] = pods['req_cpu_milli_core'].tolist()
    data['req_memory'] = pods['req_mem_mb'].tolist()
    data['items'] = list(range(len( pods)))
    data['bins'] = data['items']
    data['cpu_capacity'] = cpu
    data['memory_capacity'] = memory
    return data

In [15]:
def create_solver(data):
    x = {}
    y = {}
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')
    #solver.SetNumThreads(6)
    
   # solver.SetTimeLimit(30000)

    # Variables
    # x[i, j] = 1 if item i is packed in bin j.

    for i in data['items']:
        for j in data['bins']:
            x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

    # y[j] = 1 if bin j is used.

    for j in data['bins']:
        y[j] = solver.IntVar(0, 1, 'y[%i]' % j)
    
  
    # Constraints
    # Each item must be in exactly one bin.
    for i in data['items']:
        solver.Add(sum(x[i, j] for j in data['bins']) == 1)


    # The amount packed in each bin cannot exceed its capacity.
    for j in data['bins']:
        solver.Add(
            sum(x[(i, j)] * data['req_cpu'][i] for i in data['items']) <= y[j] * data['cpu_capacity'])


    for j in data['bins']:
        solver.Add(
            sum(x[(i, j)] * data['req_memory'][i] for i in data['items']) <= y[j] * data['memory_capacity'])
    
    
    return solver, x, y

In [16]:
def solve(solver, data, x, y, pods):
    solver.Minimize(solver.Sum([y[j] for j in data['bins']]))
    status = solver.Solve()
    solution = pd.DataFrame(columns = pods.columns, index = pods.index).sort_index().truncate(-1, -1 ).reindex()
    solution['node_name'] = "" #TODO - fix the dataframe definition to include all columns
    if status == pywraplp.Solver.OPTIMAL:
        num_bins = 0.
        for j in data['bins']:
            if y[j].solution_value() == 1:
                bin_items = []
                bin_pods = pd.DataFrame(columns = pods.columns, index = pods.index).sort_index().truncate(-1, -1 ).reindex()
                bin_cpu = 0
                bin_memory = 0
                for i in data['items']:
                    if x[i, j].solution_value() > 0:
                        bin_items.append(i)
                       # print(pods.iloc[[i]])
                        bin_pods = bin_pods.append(pods.iloc[[i]])
                        bin_pods['node_name'] = 'node' + str(j)
                        bin_cpu += data['req_cpu'][i]
                        bin_memory += data['req_memory'][i]
                if bin_cpu > 0 or bin_memory>0:
                    solution = solution.append(bin_pods.copy())
                    num_bins += 1
                    #print('Bin number', j)
                    #print('  Items packed:', bin_items)
                    #print('  Pods packed:', bin_pods)
                    #print(f"  Total cpu: {bin_cpu}, cpu utilization: {round(bin_cpu / data['cpu_capacity'] * 100, 2)}%")
                    #print(f"  Total memory: {bin_memory}, memory utilization {round(bin_memory / data['memory_capacity'] * 100, 2)}%")
                    #print()
        #print()        
        #print('Number of bins used:', num_bins)
        #print('Time = ', solver.WallTime()/1000/60, ' mins')
    else:
        print('The problem does not have an optimal solution.')
        
    return solution

In [17]:
def get_solution(curr_node, pods_data):
    global solutions
    print (f"{datetime.now().strftime('%D %H:%M:%S')}: Solving for {curr_node['API Name']} (cpu: {curr_node.cpu}, memory: {curr_node.memory})")
    
    #pod_placement = pd.DataFrame(['node_name', 'node_type', 'node_cpu', 'node_memory'] + [pods_data.columns])
    
    if (len(solutions[(solutions.cpu == curr_node.cpu) & (solutions.memory == curr_node.memory)]) == 0):
        
        data = create_data_model(curr_node.cpu, curr_node.memory, pods_data)
        solver,x,y = create_solver(data)
        pod_placement = solve(solver, data, x, y, pods_data)    
        if len(pod_placement) > 0:
            print (f"{datetime.now().strftime('%D %H:%M:%S')}: Solution for {curr_node['API Name']}: {len(pod_placement.node_name.unique())} nodes, cost: {len(pod_placement) * curr_node.cost}")
        else:
            print (f"{datetime.now().strftime('%D %H:%M:%S')}: No solution found for {curr_node['API Name']}")
    else:
        old_node = solutions[(solutions.cpu == curr_node.cpu) & (solutions.memory == curr_node.memory)].iloc[0]
        pod_placement = old_node.pod_placement.copy()        
        print (f"{datetime.now().strftime('%D %H:%M:%S')}: Reusing solution for {old_node['name']}")
            
    pod_placement['node_type'] = curr_node['API Name']
    pod_placement['node_cpu'] = curr_node.cpu
    pod_placement['node_memory'] = curr_node.memory
    
    #solutions = solutions.append({'name': curr_node['API Name'], 'cpu': curr_node['cpu'], 'memory': curr_node['memory'], 
    #                 'num_nodes': len(solution), 'cost': curr_cost, 
    #                 'solution': solution}, ignore_index = True)
    
    #if curr_cost < min_cost:
    #    best_node = nodes_data.iloc[i]
    #    min_cost = curr_cost
    #    best_solution = solution
    
    
    
    return pd.DataFrame(
        np.array([curr_node['API Name'], curr_node['cpu'], curr_node['memory'],
                len(pod_placement.node_name.unique()), curr_node.cost * len(pod_placement.node_name.unique()), pod_placement]).reshape(1,6),
        columns = solutions.columns)

In [18]:
from multiprocessing.pool import ThreadPool

solutions = pd.DataFrame(columns = ["name", "cpu", "memory", "num_nodes", "cost", "pod_placement"])


min_cost = float('inf')
best_solution = []
best_node = None

pods = pods_data#[:900]

print(f"Starting... {datetime.now().strftime('%D %H:%M:%S')}")

executor = ThreadPoolExecutor(max_workers=12)
futures = []

for i in range(1, len(nodes_data)):
    curr_node = nodes_data.iloc[i]    
    solution = get_solution(curr_node, pods)
    solutions = solutions.append(solution, ignore_index = True)

    #futures.append(executor.submit(solve_wrapper, curr_node, pods_data))
    
#for future in concurrent.futures.as_completed(futures):
        
#        to_append = pd.DataFrame(np.array(future.result()).reshape(1,6), columns = solutions.columns)
        #print(to_append.columns)
#        solutions = solutions.append(to_append, ignore_index = True)
        #print (future.result())
        
print("All solutions found")



Starting... 01/23/21 11:55:24
01/23/21 11:55:24: Solving for m5a.2xlarge (cpu: 7628, memory: 31393.923072)
01/23/21 11:55:27: Solution for m5a.2xlarge: 11 nodes, cost: 32.116
01/23/21 11:55:27: Solving for r5n.12xlarge (cpu: 47628, memory: 383393.923072)


<ipython-input-17-c7c8f4c3ba3d>:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.array([curr_node['API Name'], curr_node['cpu'], curr_node['memory'],


01/23/21 11:55:30: Solution for r5n.12xlarge: 2 nodes, cost: 333.444
01/23/21 11:55:30: Solving for r5ad.xlarge (cpu: 3628, memory: 31393.923072)
01/23/21 11:55:32: Solution for r5ad.xlarge: 22 nodes, cost: 24.42
01/23/21 11:55:32: Solving for r5n.xlarge (cpu: 3628, memory: 31393.923072)
01/23/21 11:55:32: Reusing solution for r5ad.xlarge
01/23/21 11:55:32: Solving for r5dn.xlarge (cpu: 3628, memory: 31393.923072)
01/23/21 11:55:32: Reusing solution for r5ad.xlarge
01/23/21 11:55:32: Solving for i2.xlarge (cpu: 3628, memory: 29893.923072)
01/23/21 11:55:34: Solution for i2.xlarge: 22 nodes, cost: 62.751999999999995
01/23/21 11:55:34: Solving for m5n.16xlarge (cpu: 63628, memory: 255393.923072)
01/23/21 11:55:37: Solution for m5n.16xlarge: 2 nodes, cost: 355.052
01/23/21 11:55:37: Solving for d2.8xlarge (cpu: 35628, memory: 243393.923072)
01/23/21 11:55:41: Solution for d2.8xlarge: 3 nodes, cost: 475.968
01/23/21 11:55:41: Solving for inf1.xlarge (cpu: 3628, memory: 7393.923072)
The pro

01/23/21 11:56:55: Solution for x1e.2xlarge: 11 nodes, cost: 152.144
01/23/21 11:56:55: Solving for m5dn.24xlarge (cpu: 95628, memory: 383393.923072)
01/23/21 11:56:57: Solution for m5dn.24xlarge: 1 nodes, cost: 608.724
01/23/21 11:56:57: Solving for m5a.8xlarge (cpu: 31628, memory: 127393.923072)
01/23/21 11:56:57: Reusing solution for h1.8xlarge
01/23/21 11:56:57: Solving for m5ad.2xlarge (cpu: 7628, memory: 31393.923072)
01/23/21 11:56:57: Reusing solution for m5a.2xlarge
01/23/21 11:56:57: Solving for r5a.2xlarge (cpu: 7628, memory: 63393.923072)
01/23/21 11:56:57: Reusing solution for r5n.2xlarge
01/23/21 11:56:57: Solving for m5a.12xlarge (cpu: 47628, memory: 191393.923072)
01/23/21 11:56:57: Reusing solution for m5n.12xlarge
01/23/21 11:56:57: Solving for d2.xlarge (cpu: 3628, memory: 29893.923072)
01/23/21 11:56:57: Reusing solution for i2.xlarge
01/23/21 11:56:57: Solving for m1.xlarge (cpu: 3628, memory: 14393.923072)
01/23/21 11:56:59: Solution for m1.xlarge: 22 nodes, cost:

01/23/21 11:57:32: Solution for x1e.8xlarge: 3 nodes, cost: 608.576
01/23/21 11:57:32: Solving for m6gd.16xlarge (cpu: 63628, memory: 255393.923072)
01/23/21 11:57:32: Reusing solution for m5n.16xlarge
01/23/21 11:57:32: Solving for g4ad.16xlarge (cpu: 63628, memory: 255393.923072)
01/23/21 11:57:32: Reusing solution for m5n.16xlarge
01/23/21 11:57:32: Solving for r5dn.12xlarge (cpu: 47628, memory: 383393.923072)
01/23/21 11:57:32: Reusing solution for r5n.12xlarge
01/23/21 11:57:32: Solving for r5n.4xlarge (cpu: 15628, memory: 127393.923072)
01/23/21 11:57:32: Reusing solution for r5a.4xlarge
01/23/21 11:57:32: Solving for h1.16xlarge (cpu: 63628, memory: 255393.923072)
01/23/21 11:57:32: Reusing solution for m5n.16xlarge
01/23/21 11:57:32: Solving for r5dn.4xlarge (cpu: 15628, memory: 127393.923072)
01/23/21 11:57:32: Reusing solution for r5a.4xlarge
01/23/21 11:57:32: Solving for c5ad.24xlarge (cpu: 95628, memory: 191393.923072)
01/23/21 11:57:32: Reusing solution for inf1.24xlarge


01/23/21 11:57:47: Solution for u-6tb1.metal: 1 nodes, cost: nan
01/23/21 11:57:47: Solving for m6gd.2xlarge (cpu: 7628, memory: 31393.923072)
01/23/21 11:57:47: Reusing solution for m5a.2xlarge
01/23/21 11:57:47: Solving for f1.16xlarge (cpu: 63628, memory: 975393.923072)
01/23/21 11:57:47: Reusing solution for x1.16xlarge
01/23/21 11:57:47: Solving for u-9tb1.metal (cpu: 447628, memory: 9215393.923072)
01/23/21 11:57:49: Solution for u-9tb1.metal: 1 nodes, cost: nan
01/23/21 11:57:49: Solving for c6g.8xlarge (cpu: 31628, memory: 63393.923072)
01/23/21 11:57:49: Reusing solution for c5a.8xlarge
01/23/21 11:57:49: Solving for g3s.xlarge (cpu: 3628, memory: 29893.923072)
01/23/21 11:57:49: Reusing solution for i2.xlarge
01/23/21 11:57:49: Solving for r5b.xlarge (cpu: 3628, memory: 31393.923072)
01/23/21 11:57:49: Reusing solution for r5ad.xlarge
01/23/21 11:57:49: Solving for a1.4xlarge (cpu: 15628, memory: 31393.923072)
01/23/21 11:57:49: Reusing solution for a1.metal
01/23/21 11:57:49

01/23/21 11:58:34: Solution for c5n.9xlarge: 3 nodes, cost: 181.3
01/23/21 11:58:34: Solving for c5.xlarge (cpu: 3628, memory: 7393.923072)
01/23/21 11:58:34: Reusing solution for inf1.xlarge
01/23/21 11:58:34: Solving for g4dn.metal (cpu: 95628, memory: 383393.923072)
01/23/21 11:58:34: Reusing solution for m5dn.24xlarge
01/23/21 11:58:34: Solving for d3en.12xlarge (cpu: 47628, memory: 191393.923072)
01/23/21 11:58:34: Reusing solution for m5n.12xlarge
01/23/21 11:58:34: Solving for d3.4xlarge (cpu: 15628, memory: 127393.923072)
01/23/21 11:58:34: Reusing solution for r5a.4xlarge
01/23/21 11:58:34: Solving for d3.2xlarge (cpu: 7628, memory: 63393.923072)
01/23/21 11:58:34: Reusing solution for r5n.2xlarge
01/23/21 11:58:34: Solving for m5zn.12xlarge (cpu: 47628, memory: 191393.923072)
01/23/21 11:58:34: Reusing solution for m5n.12xlarge
01/23/21 11:58:34: Solving for m5d.4xlarge (cpu: 15628, memory: 63393.923072)
01/23/21 11:58:34: Reusing solution for h1.4xlarge
01/23/21 11:58:34: So

## Add Daemonset back to Solution

In [19]:

for i in range(0, len(solutions) -1):
    all_nodes = solutions.iloc[i].pod_placement.groupby(['node_name', 'node_type', 'node_cpu', 'node_memory']).size().\
            reset_index().iloc[:,0:4]  
    
    daemonset_placement = all_nodes.merge(daemonset.reset_index(), how='cross', left_on=None, right_on=None)
    
    solutions.iloc[i].pod_placement  = solutions.iloc[i].pod_placement.append(daemonset_placement)


In [20]:
#solutions[solutions.columns[:-1]]
len(solutions)
#display(solutions[["name", "cpu", "memory", "num_nodes", "cost"]].sort_values(by = "cost",
#                    ascending = "False").reset_index(drop = True))

solutions[["name", "cpu", "memory", "num_nodes", "cost"]].sort_values(by = "cost",
                    ascending = "False").reset_index(drop = True).to_csv("solutions.csv", index=False, )


solutions = solutions[solutions.cost > 0].sort_values(by = "cost", axis=0,
                    ascending = "False").reset_index(drop = True)


In [21]:
best_solution = solutions.iloc[0]

print(f"Best solution: \n")    
print(f"Node: {best_solution['name']}, cpu: {best_solution.cpu},\
          memory: {best_solution.memory}, hourly cost: {best_solution.cost}, number of nodes: {len(best_solution.pod_placement.node_name.unique())}")


#display(best_solution.pod_placement)

best_solution.pod_placement.to_csv('best_solution.csv')


Best solution: 

Node: a1.metal, cpu: 15628,          memory: 31393.923072, hourly cost: 1.542, number of nodes: 6


In [22]:
all_placements = solutions.iloc[0].pod_placement.copy()

for i in range(1, len(solutions)):
    all_placements = all_placements.append( solutions.pod_placement.iloc[i].copy())
    
all_placements.to_csv("all_solutions.csv")

## Stats
#20 nodes - 1 hour with 6 threads
#20 nodes - 1 hour with 3 threads
#20 nodes - 1 hour with 12 threads


#Number of Pods
#200 pods  7 sec
#400 pods - 64 sec
#600 pods - 9 mins
#800 pods - 36 mins
#1000

#2000 pods - stopped after 18 hours?


#on parallelism https://github.com/google/or-tools/issues/1656